# Importing Data

In [131]:
import pandas as pd
medal_host          = pd.read_csv(r"C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\Clean_Medal_Host_df.csv")
team_data           = pd.read_csv(r"C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\Nation's Team Data by Year.csv")
program_data        = pd.read_csv(r"C:\Users\lemon\OneDrive\Desktop\Github Files\Mathmatical Competition in Modeling (2025) Problem C\File for Github Upload (Code is Broken Down)\summerOly_programs.csv", encoding='latin-1')

In [132]:
team_data = team_data.fillna(0)
team_data.head()

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Total_Athletes_in_Team,Gold_Winning_Athletes,Silver_Winning_Athletes,Bronze_Winning_Athletes,Athletes_Who_Returned,Multi_Event_Athletes,Average Team Rating
0,1,USA,37,24,32,93,2000,586.0,99.0,51.0,30.0,197.0,155.0,5.167064
1,3,CHN,28,16,14,58,2000,266.0,23.0,9.0,12.0,97.0,102.0,4.691734
2,4,AUS,16,25,17,58,2000,617.0,46.0,47.0,45.0,199.0,150.0,4.523068
3,5,DEU,13,17,26,56,2000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,6,FRA,13,14,11,38,2000,336.0,14.0,21.0,12.0,110.0,96.0,4.276229


In [133]:
#TRAINING DATA (already 2000-2024)
training_df = team_data[team_data['Year'] < 2024]

#TESTING DATA 
test_df = team_data[team_data['Year'] == 2024]

In [134]:
print('Train NOC: ', training_df['NOC'].nunique())
print('Test NOC: ', test_df['NOC'].nunique())

Train NOC:  115
Test NOC:  86


In [135]:
noc_to_keep = test_df['NOC'].unique()
filtered_df = team_data[team_data["NOC"].isin(noc_to_keep)]
filtered_df

,Rank,NOC,Gold,Silver,Bronze,Total,Year,Total_Athletes_in_Team,Gold_Winning_Athletes,Silver_Winning_Athletes,Bronze_Winning_Athletes,Athletes_Who_Returned,Multi_Event_Athletes,Average Team Rating
0,1,USA,37,24,32,93,2000,586.0,99.0,51.0,30.0,197.0,155.0,5.167064
1,3,CHN,28,16,14,58,2000,266.0,23.0,9.0,12.0,97.0,102.0,4.691734
2,4,AUS,16,25,17,58,2000,617.0,46.0,47.0,45.0,199.0,150.0,4.523068
3,5,DEU,13,17,26,56,2000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,6,FRA,13,14,11,38,2000,336.0,14.0,21.0,12.0,110.0,96.0,4.276229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
546,84,PER,0,0,1,1,2024,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
547,84,QAT,0,0,1,1,2024,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
548,84,SGP,0,0,1,1,2024,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
549,84,SVK,0,0,1,1,2024,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [136]:
#TRAINING DATA (already 2000-2024)
training_df = filtered_df[filtered_df['Year'] < 2024]

#TESTING DATA 
test_df = filtered_df[filtered_df['Year'] == 2024]

print('Train NOC: ', training_df['NOC'].nunique())
print('Test NOC: ', test_df['NOC'].nunique())

Train NOC:  79
Test NOC:  86


There are 7 new teams participating in the 2024 olympics, some of which did earn medals. The teams may be from newly established countries or have created new teams after being a part of a joint team in the past.

In [137]:
set1 = set(training_df['NOC'].unique())
set2 = set(test_df['NOC'].unique())

# Find the new teams in test data (2024 Olympics) that are not in the training data (2000-2020 Olympics)
new_teams_list = set2-set1

# Make a DF for those teams so that later, we can add their total number of medals won to the rest of our team predictions.
new_teams_df = team_data[team_data["NOC"].isin(new_teams_list)]

# Example of Weighted Least Squared Reg. in Python
**Vistit https://github.com/Kenzo-Hoobert/MCM-ICM-2025 for the modeling example and work along.** 

In [138]:
import numpy as np
import statsmodels.api as sm

# Ensure NOC is the index
training_df.set_index('NOC', inplace=True)

# Features (X) and Target (y) from training data
X_train = training_df.drop(columns=['Rank','Gold', 'Silver', 'Bronze', 'Total'])  # Exclude target columns
y_train = training_df['Total']

# Get years for weighting
years = training_df['Year']

# Calculate weights (higher weight for recent years)
alpha = 0.75
weights = np.exp(-alpha * (2020 - years))

# Fit Weighted Least Squares (WLS) model
X_train = sm.add_constant(X_train)  # Add intercept
model = sm.WLS(y_train, X_train, weights=weights)
results = model.fit()

# Print model summary
print(results.summary())


                            WLS Regression Results                            
Dep. Variable:                  Total   R-squared:                       0.800
Model:                            WLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     188.8
Date:                Tue, 11 Feb 2025   Prob (F-statistic):          8.45e-127
Time:                        16:14:15   Log-Likelihood:                -2435.0
No. Observations:                 386   AIC:                             4888.
Df Residuals:                     377   BIC:                             4924.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    3